# Assignment 3 (Concise, without reading data from external source)


$$ \sum_{i=A,...G,j=1,..,4} (prod\_cost_{i,j}+trans\_cost_{i,j})x_{i,j}\\
s.t. \\
Demand: \sum_{i=A,...,G}x_{ij} \ge Demand_i,  j=1,2,3,4\\
Capacity: \sum_{j=1,...,4}proc\_time_{i,j}x_{i,j}\le i=A,...,G \\
A1: x_{A,1}=0 \\
D1: x_{D,1}=0 \\
$$

## Import PuLP modeler functions

In [8]:
from pulp import *
#import pandas as pd
#import numpy as np

## Original Problem

In [9]:
probA=LpProblem("Problem A",LpMinimize)

factory=['A','B','C','D','E','F','G']

yarn = ['1','2','3','4']

# Creates a list of production cost
prod_cost =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]

prod_cost= makeDict([factory,yarn],prod_cost)

# Creates a list of transportation cost
trans_cost = [
    [ 0.30,0.30,0.45, 0.45],\
    [ 0.40,0.40,0.60,0.60],\
    [ 0.80,0.80, 1.20,1.20],\
    [ 0.70,0.70,1.05,1.05],\
    [ 0.70, 0.70, 1.05, 1.05],\
    [ 0,0,0,0],\
    [ 0.50, 0.50, 0.75, 0.75]]

trans_cost= makeDict([factory,yarn],trans_cost)

proc_time=[
    [0, 0.400,0.375,0.250],\
    [0.700,0.500,0.350,0.250],\
    [0.675,0.450,0.400,0.250],\
    [0,0.450,0.350,0.200],\
    [0.650,0.450,0.400,0.250],\
    [0.625,0.500,0.425,0.425],\
    [0.700,0.450,0.350,0.400],\
]
proc_time= makeDict([factory,yarn],proc_time)
capacity = {
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500}
demand={
    "1":25000,
    "2":26000,
    "3":28000,
    "4":28000
}

production_vars = LpVariable.dicts("x", (factory,yarn),lowBound=0, cat='Continuous')

In [10]:
#objective function
probA+=lpSum([production_vars[i][j]*(prod_cost[i][j]+trans_cost[i][j]) for i in factory for j in yarn]),"total cost"
# demand constraint
for j in demand:
    probA += lpSum([production_vars[i][j] for i in factory ]) >= demand[j],"demand_%s"%j

# capacity constraint    
for i in factory:
    probA += lpSum([proc_time[i][j]*production_vars[i][j] for j in yarn]) <= capacity[i],"capacity_%s"%i

# x_1A and x_1D constraint
probA += production_vars["A"]["1"]==0,"xA1=0"
probA += production_vars["D"]["1"]==0,"xD1=0"

In [11]:
probA.writeLP("Filatoi2A.lp")
probA.solve()
print("Status:",LpStatus[probA.status])
#print optimization result
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probA.objective))
original_cost=value(probA.objective)

Status: Optimal
x_A_1 = 0.0 	Reduced Cost = 0.0
x_A_2 = 6250.0 	Reduced Cost = 0.0
x_A_3 = 0.0 	Reduced Cost = 0.35514706
x_A_4 = 0.0 	Reduced Cost = 1.2867647
x_B_1 = 4285.7143 	Reduced Cost = 0.0
x_B_2 = 0.0 	Reduced Cost = 0.80798319
x_B_3 = 0.0 	Reduced Cost = 0.38676471
x_B_4 = 0.0 	Reduced Cost = 0.88340336
x_C_1 = 3703.7037 	Reduced Cost = 0.0
x_C_2 = 0.0 	Reduced Cost = 0.97686275
x_C_3 = 0.0 	Reduced Cost = 0.71394336
x_C_4 = 0.0 	Reduced Cost = 1.4087146
x_D_1 = 0.0 	Reduced Cost = 0.0
x_D_2 = 0.0 	Reduced Cost = 0.041176471
x_D_3 = 2040.1255 	Reduced Cost = 0.0
x_D_4 = 0.0 	Reduced Cost = 0.85
x_E_1 = 3846.1538 	Reduced Cost = -2.220446e-16
x_E_2 = 0.0 	Reduced Cost = 0.49208145
x_E_3 = 0.0 	Reduced Cost = 0.99524887
x_E_4 = 0.0 	Reduced Cost = 1.3782805
x_F_1 = 13164.428 	Reduced Cost = 0.0
x_F_2 = 19750.0 	Reduced Cost = 0.0
x_F_3 = 18817.017 	Reduced Cost = 7.7715612e-16
x_F_4 = 28000.0 	Reduced Cost = -9.9920072e-16
x_G_1 = 0.0 	Reduced Cost = 2.2764706
x_G_2 = 0.0 	Redu

## Additional Sensitivity Question 1
- A client has just called asking for an additional 5,000 kg of the **medium size** yarn. The original demand still has to be met but Filatoi is considering whether it should accept the new order and how much it should charge for it.
- Give your answer assuming Filatoi **cannot** adjust down the outsourcing orders it has already committed to with the other suppliers. 
- **change demand[3] from 28000 to 28000+5000=33000**


In [12]:
probS1=LpProblem("Problem Sensitivity 1",LpMinimize)

factory=['A','B','C','D','E','F','G']

yarn = ['1','2','3','4']

# Creates a list of production cost
prod_cost =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]

prod_cost= makeDict([factory,yarn],prod_cost)

# Creates a list of transportation cost
trans_cost = [
    [ 0.30,0.30,0.45, 0.45],\
    [ 0.40,0.40,0.60,0.60],\
    [ 0.80,0.80, 1.20,1.20],\
    [ 0.70,0.70,1.05,1.05],\
    [ 0.70, 0.70, 1.05, 1.05],\
    [ 0,0,0,0],\
    [ 0.50, 0.50, 0.75, 0.75]]

trans_cost= makeDict([factory,yarn],trans_cost)

proc_time=[
    [0, 0.400,0.375,0.250],\
    [0.700,0.500,0.350,0.250],\
    [0.675,0.450,0.400,0.250],\
    [0,0.450,0.350,0.200],\
    [0.650,0.450,0.400,0.250],\
    [0.625,0.500,0.425,0.425],\
    [0.700,0.450,0.350,0.400],\
]
proc_time= makeDict([factory,yarn],proc_time)
capacity = {
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500}
addtionalD3=5000
demand={
    "1":25000,
    "2":26000,
    "3":28000+addtionalD3,
    "4":28000
}

production_vars_s1 = LpVariable.dicts("x", (factory,yarn),lowBound=0, cat='Continuous')

### Additioal Sensitivity Question 1: Add objective function and constraints
- Use lpSum() amd += operator to construct objective function and constraintsumn based modelling: relates to the variable’s existence in the objective function and constraints

In [13]:
#objective function
probS1+=lpSum([production_vars_s1[i][j]*(prod_cost[i][j]+trans_cost[i][j]) for i in factory for j in yarn]),"total cost"
# demand constraint
for j in demand:
    probS1 += lpSum([production_vars_s1[i][j] for i in factory ]) >= demand[j],"demand_%s"%j

# capacity constraint    
for i in factory:
    probS1 += lpSum([proc_time[i][j]*production_vars_s1[i][j] for j in yarn]) <= capacity[i],"capacity_%s"%i

# x_1S and x_1D constraint
probS1 += production_vars_s1["A"]["1"]==0,"xA1=0"
probS1 += production_vars_s1["D"]["1"]==0,"xD1=0"

### Additioal Sensitivity Question 1: Run solver

- use name.solve(solver=None), where name is the LP problem variable defined by LpProblem
- Solve the given Lp problem. 
- This function changes the problem to make it suitable for solving then calls the solver.actualSolve method to find the solution. 
- solver – Optional: the specific solver to be used, defaults to the default solver.

In [14]:
probS1.writeLP("Filatoi_s1.lp")
probS1.solve()
print("Status:",LpStatus[probS1.status])

Status: Optimal


### Additioal Sensitivity Question 1: Print the optiomal solution

In [15]:
for v in probS1.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probS1.objective))
new_cost=value(probS1.objective)

x_A_1 = 0.0 	Reduced Cost = 0.0
x_A_2 = 6250.0 	Reduced Cost = 0.0
x_A_3 = 0.0 	Reduced Cost = 0.35514706
x_A_4 = 0.0 	Reduced Cost = 1.2867647
x_B_1 = 4285.7143 	Reduced Cost = 0.0
x_B_2 = 0.0 	Reduced Cost = 0.80798319
x_B_3 = 0.0 	Reduced Cost = 0.38676471
x_B_4 = 0.0 	Reduced Cost = 0.88340336
x_C_1 = 3703.7037 	Reduced Cost = 0.0
x_C_2 = 0.0 	Reduced Cost = 0.97686275
x_C_3 = 0.0 	Reduced Cost = 0.71394336
x_C_4 = 0.0 	Reduced Cost = 1.4087146
x_D_1 = 0.0 	Reduced Cost = 0.0
x_D_2 = 0.0 	Reduced Cost = 0.041176471
x_D_3 = 7040.1255 	Reduced Cost = 0.0
x_D_4 = 0.0 	Reduced Cost = 0.85
x_E_1 = 3846.1538 	Reduced Cost = -2.220446e-16
x_E_2 = 0.0 	Reduced Cost = 0.49208145
x_E_3 = 0.0 	Reduced Cost = 0.99524887
x_E_4 = 0.0 	Reduced Cost = 1.3782805
x_F_1 = 13164.428 	Reduced Cost = 0.0
x_F_2 = 19750.0 	Reduced Cost = 0.0
x_F_3 = 18817.017 	Reduced Cost = 7.7715612e-16
x_F_4 = 28000.0 	Reduced Cost = -9.9920072e-16
x_G_1 = 0.0 	Reduced Cost = 2.2764706
x_G_2 = 0.0 	Reduced Cost = 0.469

In [16]:
for u,v in zip(probA.variables(),probS1.variables()):
    print(v.name,"=",v.varValue-u.varValue)
    
print("(new cost - original cost)/5000 =",(new_cost-original_cost)/5000)

x_A_1 = 0.0
x_A_2 = 0.0
x_A_3 = 0.0
x_A_4 = 0.0
x_B_1 = 0.0
x_B_2 = 0.0
x_B_3 = 0.0
x_B_4 = 0.0
x_C_1 = 0.0
x_C_2 = 0.0
x_C_3 = 0.0
x_C_4 = 0.0
x_D_1 = 0.0
x_D_2 = 0.0
x_D_3 = 5000.0
x_D_4 = 0.0
x_E_1 = 0.0
x_E_2 = 0.0
x_E_3 = 0.0
x_E_4 = 0.0
x_F_1 = 0.0
x_F_2 = 0.0
x_F_3 = 0.0
x_F_4 = 0.0
x_G_1 = 0.0
x_G_2 = 0.0
x_G_3 = 0.0
x_G_4 = 0.0
(new cost - original cost)/5000 = 12.3


In [17]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probS1.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)


Sensitivity Analysis
Constraint		Shadow Price	Slack
demand_1 : x_A_1 + x_B_1 + x_C_1 + x_D_1 + x_E_1 + x_F_1 + x_G_1 >= 25000 	 19.573529 		 -0.0
demand_2 : x_A_2 + x_B_2 + x_C_2 + x_D_2 + x_E_2 + x_F_2 + x_G_2 >= 26000 	 14.958824 		 -0.0
demand_3 : x_A_3 + x_B_3 + x_C_3 + x_D_3 + x_E_3 + x_F_3 + x_G_3 >= 33000 	 12.3 		 -0.0
demand_4 : x_A_4 + x_B_4 + x_C_4 + x_D_4 + x_E_4 + x_F_4 + x_G_4 >= 28000 	 9.8 		 -0.0
capacity_A : 0.4*x_A_2 + 0.375*x_A_3 + 0.25*x_A_4 <= 2500.0 	 -4.1470588 		 -0.0
capacity_B : 0.7*x_B_1 + 0.5*x_B_2 + 0.35*x_B_3 + 0.25*x_B_4 <= 3000.0 	 -2.5336134 		 -0.0
capacity_C : 0.675*x_C_1 + 0.45*x_C_2 + 0.4*x_C_3 + 0.25*x_C_4 <= 2500.0 	 -2.0348584 		 -0.0
capacity_D : 0.45*x_D_2 + 0.35*x_D_3 + 0.2*x_D_4 <= 2600.0 	 0.0 		 135.95609999999988
capacity_E : 0.65*x_E_1 + 0.45*x_E_2 + 0.4*x_E_3 + 0.25*x_E_4 <= 2500.0 	 -2.1131222 		 -0.0
capacity_F : 0.625*x_F_1 + 0.5*x_F_2 + 0.425*x_F_3 + 0.425*x_F_4 <= 38000.0 	 -2.1176471 		 -0.0
capacity_G : 0.7*x_G_1 + 0.45*x_G_2 + 